In [0]:
import geopandas as gpd
import shutil
import os
from pyspark.sql.functions import expr

In [0]:
catalog_dev = "`land_topografisk-gdb_dev`"
schema_dev = "ai2025"
gold_table = f"{catalog_dev}.{schema_dev}.predicted_snuplasser_gold"

In [0]:
catalov_eiendom = "eiendom_smia_dev"
schema_eiendom = "admin_enheter_silver"
kommune_table = f"{catalov_eiendom}.{schema_eiendom}.kommune"
fylke_table = f"{catalov_eiendom}.{schema_eiendom}.fylke"
srid = 25833

In [0]:
def get_kommune_and_fylke():
    """
    Returnerer en Spark DataFrame med kolonner 'kommunenummer', 'kommunenavn', 'fylkesnummer', 'fylkesnavn'.
    """
    predicted_gold_df = spark.read.table(gold_table).withColumn(
        "geometry", expr(f"ST_SetSRID(ST_GeomFromWKB(geometry), {srid})")
    )
    kom_df = spark.read.table(kommune_table).withColumn(
        "kom_geom", expr(f"ST_SetSRID(ST_GeomFromWKB(geometry), {srid})")
    )
    fylke_df = spark.read.table(fylke_table)

    predicted_gold_df.createOrReplaceTempView("predicted")
    kom_df.createOrReplaceTempView("kommune")
    fylke_df.createOrReplaceTempView("fylke")

    joined_df = spark.sql(
        """
        SELECT p.*, poly_k.kommunenummer,  poly_k.kommunenavn, poly_f.fylkesnummer, poly_f.fylkesnavn
        FROM predicted p, kommune poly_k, fylke poly_f
        WHERE ST_Contains(poly_k.kom_geom, p.geometry)
        AND poly_f.fylkesnummer = LEFT(poly_k.kommunenummer, 2)
    """
    )

    return joined_df

In [0]:
def to_geopackage():
    """
    Lagrer GeoDataFrame til GeoPackage-fil.
    """
    local_gpkg = os.path.join(os.getcwd(), "predicted_snuplasser.gpkg")
    final_gpkg_dir = "/Volumes/land_topografisk-gdb_dev/external_dev/static_data/DL_SNUPLASSER/geopackages/"
    final_gpkg_path = os.path.join(final_gpkg_dir, "predicted_snuplasser.gpkg")

    joined_df = get_kommune_and_fylke()
    pdf = joined_df.toPandas()
    gdf = gpd.GeoDataFrame(pdf, geometry="geometry")
    gdf.set_crs(epsg=25833, inplace=True)

    categories = gdf["category"].unique()  # 20-25m, 25-30m, 30-35m, 35m+

    for i, cat in enumerate(categories):
        layer_name = f"predicted_{cat}"

        mode = "w" if i == 0 else "a"

        gdf[gdf["category"] == cat].to_file(
            local_gpkg, driver="GPKG", layer=layer_name, mode=mode
        )

    # Kopierer fra lokalt til katalogen
    shutil.copyfile(local_gpkg, final_gpkg_path)

    # Sletter lokal fil
    os.remove(local_gpkg)

In [0]:
to_geopackage()